In [1]:

import jax
jax.config.update('jax_platforms', 'cpu')
import bmi
from src.minde.minde_cond import Minde_c
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from scripts.helper import get_data_loader, get_samples
from argparse import Namespace


/home/bounoua/work/minde/myvenv/lib/python3.9/site-packages/jaxtyping/__init__.py:244: UserWarning: jaxtyping version >=0.2.23 should be used with Equinox version >=0.11.1
  warnings.warn(


In [2]:
#bmi.benchmark.BENCHMARK_TASKS
name_task = "1v1-normal-0.75"
task = bmi.benchmark.BENCHMARK_TASKS[name_task]

In [3]:

args = Namespace(preprocessing="rescale", 
                 Train_Size=100000,
                 Test_Size = 10000,
                 seed = 42,
                 num_epoch = 100,
                 batch_size="256",
                 lr=1e-2 ,
                 importance_sampling_train= True,
                 importance_sampling_test = True,
                 test_epoch = 1,
                 lr = 0.01,
                 bs = 256,
                 use_ema = True,
                 max_epochs = 1000,
                accelerator="gpu",
                outdir = "/logs/"
                 )
train_l,test_l = get_data_loader(args,task)


[1.]


In [4]:
pl.seed_everything(args.seed)

minde = Minde_c(args,
                dim_x=task.dim_x,
                dim_y=task.dim_y,
                gt=task.mutual_information,
               )

minde.fit(train_l,test_l)

minde.compute_mi()



/home/bounoua/miniconda3/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/bounoua/miniconda3/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torc

Sanity Checking: |                                                                                            …

/home/bounoua/miniconda3/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |                                                                                                   …

/home/bounoua/miniconda3/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

Validation: |                                                                      | 0/? [00:00<?, ?it/s]

Step :6248 , GT:0.41333928659223396, MINDE_C :0.3701082468032837 , MINDE_C_(sigma=1) :0.3702295124530792    


Validation: |                                                                      | 0/? [00:00<?, ?it/s]

Step :7029 , GT:0.41333928659223396, MINDE_C :0.3787267506122589 , MINDE_C_(sigma=1) :0.36129406094551086    


/home/bounoua/work/minde/myvenv/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
